In [ ]:
import time
start = time.time()
import os
os.chdir("/Users/yogesh/Downloads/hotstar_dataset/")
import pandas as pd


df = pd.read_pickle("train_data.pkl")
df = df.T
df = df.iloc[:1000]
import warnings
warnings.filterwarnings("ignore")
df["index"] = df.index
df_500 = df.copy()
df_500["user"] = df_500["index"].str.replace("train-","")
df_500["segment"] = df_500["segment"].str.replace("neg","0")
df_500["segment"] = df_500["segment"].str.replace("pos","0")

print ("1st level")
df_500 = df_500.drop(["index","titles"],1)
to_be_recommended = df_500.iloc[800:]
#to_be_recommended = to_be_recommended.drop(["genres"],1)
df_500 = df_500[:800]

cities = df_500["cities"].values.tolist()

final=[]
for i in range(len(cities)):
    frames=[]
    user_cities = cities[i].split(",")
    flat_list = [item for sublist in [i.split(':', 1) for i in user_cities] for item in sublist]
    city = pd.DataFrame(flat_list[0::2]).T
    second = pd.DataFrame(flat_list[1::2]).T
    concatenated_df = pd.concat([city,second],ignore_index=True)
    concatenated_df.columns = concatenated_df.iloc[0]
    concatenated_df = concatenated_df.drop([0],0)
    frames.append(concatenated_df)
    final_df = pd.concat(frames,join='outer', ignore_index=True)
    final.append(pd.DataFrame(final_df))
    
city = pd.concat(final,join='outer', ignore_index=True)
city = city.fillna(0) 

genres = df_500["genres"].values.tolist()
final_1=[]
for i in range(len(genres)):
    frames_genres=[]
    genre = genres[i].split(",")
    flat_list_g = [item for sublist in [i.split(':', 1) for i in genre] for item in sublist]
    flat_list_g = [i.lower() for i in flat_list_g]
    genre_1 = pd.DataFrame(flat_list_g[0::2]).T
    second_g = pd.DataFrame(flat_list_g[1::2]).T
    genre_df = pd.concat([genre_1,second_g],ignore_index = True)
    genre_df.columns = genre_df.iloc[0]
    genre_df = genre_df.drop([0],0)
    frames_genres.append(genre_df)
    final_df_g = pd.concat(frames_genres,join='outer', ignore_index=True)
    final_1.append(pd.DataFrame(final_df_g))
    
genre = pd.concat(final_1,join='outer', ignore_index=True)
genre = genre.fillna(0) 

days_of_week = df_500["dow"].values.tolist()
final_2=[]
for i in range(len(days_of_week)):
    frames=[]
    user_days = days_of_week[i].split(",")
    flat_list = [item for sublist in [i.split(':', 1) for i in user_days] for item in sublist]
#    flat_list = [int(i) for i in flat_list]
    day = pd.DataFrame(flat_list[0::2]).T
    second = pd.DataFrame(flat_list[1::2]).T
    concatenated_df = pd.concat([day,second],ignore_index=True)
    concatenated_df.columns = concatenated_df.iloc[0]
    concatenated_df = concatenated_df.drop([0],0)
    frames.append(concatenated_df)
    final_df = pd.concat(frames,join='outer', ignore_index=True)
    final_2.append(pd.DataFrame(final_df))

days = pd.concat(final_2,join='outer', ignore_index=True)
days = days.fillna(0)
days_dict = {"1":"Sunday", "2":"Monday", "3":"Tuesday", "4":"Wednesday", "5":"Thursday", 
             "6":"Friday", "7":"Saturday"} 
day_key = pd.DataFrame(list(days_dict.items()),columns = ["dow_cat","day"])  
d = day_key.set_index('dow_cat').to_dict()  
days.columns = days.columns.to_series().map(d['day'])
print ("Level 2")
time_am = [str(x) for x in range(0,12)]
time_pm=[str(x) for x in range(12,24)]
am=[str(x)+ " am" for x in range(0,12)]
pm=[str(x)+ " pm" for x in range(0,12)]
d1 = dict(zip(time_am,am))
d2 = dict(zip(time_pm,pm))
d1["0"] = "12 am" #changing 0 to 12 am
d2["12"] = "12 pm" #changing 12 to 12 pm 
time_dict = {**d1,**d2} #merging both dicts

time_of_day = df_500["tod"].values.tolist()
final_3=[]
for i in range(len(days_of_week)):
    frames=[]
    user_time = time_of_day[i].split(",")
    flat_list = [item for sublist in [i.split(':', 1) for i in user_time] for item in sublist]
#    flat_list = [int(i) for i in flat_list]
    time = pd.DataFrame(flat_list[0::2]).T
    second = pd.DataFrame(flat_list[1::2]).T
    concatenated_df = pd.concat([time,second],ignore_index=True)
    concatenated_df.columns = concatenated_df.iloc[0]
    concatenated_df = concatenated_df.drop([0],0)
    frames.append(concatenated_df)
    final_df = pd.concat(frames,join='outer', ignore_index=True)
    final_3.append(pd.DataFrame(final_df))
timeofday = pd.concat(final_3,join='outer', ignore_index=True)
timeofday = timeofday.fillna(0)
time_key = pd.DataFrame(list(time_dict.items()),columns = ["tod_cat","hour"]) 
t = time_key.set_index('tod_cat').to_dict() 
timeofday.columns = timeofday.columns.to_series().map(t['hour']) 
time_of_day = timeofday

df_500 = df_500.reset_index()
train = pd.concat([df_500,city,genre,days,time_of_day],axis = 1)
drop = ['cities', 'dow', 'genres','tod','index']
train = train.drop(drop,1)
train = train.apply(pd.to_numeric)

print ("Level 3")
cities = to_be_recommended["cities"].values.tolist()

final=[]
for i in range(len(cities)):
    frames=[]
    user_cities = cities[i].split(",")
    flat_list = [item for sublist in [i.split(':', 1) for i in user_cities] for item in sublist]
    city = pd.DataFrame(flat_list[0::2]).T
    second = pd.DataFrame(flat_list[1::2]).T
    concatenated_df = pd.concat([city,second],ignore_index=True)
    concatenated_df.columns = concatenated_df.iloc[0]
    concatenated_df = concatenated_df.drop([0],0)
    frames.append(concatenated_df)
    final_df = pd.concat(frames,join='outer', ignore_index=True)
    final.append(pd.DataFrame(final_df))
    
city = pd.concat(final,join='outer', ignore_index=True)
city = city.fillna(0) 

genres = to_be_recommended["genres"].values.tolist()
final_1=[]
for i in range(len(genres)):
    frames_genres=[]
    genre = genres[i].split(",")
    flat_list_g = [item for sublist in [i.split(':', 1) for i in genre] for item in sublist]
    flat_list_g = [i.lower() for i in flat_list_g]
    genre_1 = pd.DataFrame(flat_list_g[0::2]).T
    second_g = pd.DataFrame(flat_list_g[1::2]).T
    genre_df = pd.concat([genre_1,second_g],ignore_index = True)
    genre_df.columns = genre_df.iloc[0]
    genre_df = genre_df.drop([0],0)
    frames_genres.append(genre_df)
    final_df_g = pd.concat(frames_genres,join='outer', ignore_index=True)
    final_1.append(pd.DataFrame(final_df_g))
    
genre = pd.concat(final_1,join='outer', ignore_index=True)
genre = genre.fillna(0) 

days_of_week = to_be_recommended["dow"].values.tolist()
final_2=[]
for i in range(len(days_of_week)):
    frames=[]
    user_days = days_of_week[i].split(",")
    flat_list = [item for sublist in [i.split(':', 1) for i in user_days] for item in sublist]
#    flat_list = [int(i) for i in flat_list]
    day = pd.DataFrame(flat_list[0::2]).T
    second = pd.DataFrame(flat_list[1::2]).T
    concatenated_df = pd.concat([day,second],ignore_index=True)
    concatenated_df.columns = concatenated_df.iloc[0]
    concatenated_df = concatenated_df.drop([0],0)
    frames.append(concatenated_df)
    final_df = pd.concat(frames,join='outer', ignore_index=True)
    final_2.append(pd.DataFrame(final_df))

days = pd.concat(final_2,join='outer', ignore_index=True)
days = days.fillna(0)
days_dict = {"1":"Sunday", "2":"Monday", "3":"Tuesday", "4":"Wednesday", "5":"Thursday", 
             "6":"Friday", "7":"Saturday"} 
day_key = pd.DataFrame(list(days_dict.items()),columns = ["dow_cat","day"])  
d = day_key.set_index('dow_cat').to_dict()  
days.columns = days.columns.to_series().map(d['day'])

time_am = [str(x) for x in range(0,12)]
time_pm=[str(x) for x in range(12,24)]
am=[str(x)+ " am" for x in range(0,12)]
pm=[str(x)+ " pm" for x in range(0,12)]
d1 = dict(zip(time_am,am))
d2 = dict(zip(time_pm,pm))
d1["0"] = "12 am" #changing 0 to 12 am
d2["12"] = "12 pm" #changing 12 to 12 pm 
time_dict = {**d1,**d2} #merging both dicts

time_of_day = to_be_recommended["tod"].values.tolist()
final_3=[]
for i in range(len(days_of_week)):
    frames=[]
    user_time = time_of_day[i].split(",")
    flat_list = [item for sublist in [i.split(':', 1) for i in user_time] for item in sublist]
#    flat_list = [int(i) for i in flat_list]
    time = pd.DataFrame(flat_list[0::2]).T
    second = pd.DataFrame(flat_list[1::2]).T
    concatenated_df = pd.concat([time,second],ignore_index=True)
    concatenated_df.columns = concatenated_df.iloc[0]
    concatenated_df = concatenated_df.drop([0],0)
    frames.append(concatenated_df)
    final_df = pd.concat(frames,join='outer', ignore_index=True)
    final_3.append(pd.DataFrame(final_df))
timeofday = pd.concat(final_3,join='outer', ignore_index=True)
timeofday = timeofday.fillna(0)
time_key = pd.DataFrame(list(time_dict.items()),columns = ["tod_cat","hour"]) 
t = time_key.set_index('tod_cat').to_dict() 
timeofday.columns = timeofday.columns.to_series().map(t['hour']) 
time_of_day = timeofday

to_be_recommended = to_be_recommended.reset_index()

test = pd.concat([to_be_recommended,city,genre,days,time_of_day],axis = 1)
drop = ['cities', 'dow', "index","genres",'tod']
test = test.drop(drop,1)
test= test.apply(pd.to_numeric)

train = train.iloc[:]
test = test[:]
from sklearn.metrics.pairwise import cosine_similarity
cols = train["user"].values.tolist()
cols = ["user "+ str(x) for x in cols]
corr_matrix = pd.DataFrame(cosine_similarity(train),columns = cols)

new_user = test.iloc[19]
#39,23,1,11,19
new_user = pd.DataFrame(new_user)
new_user = new_user.T
new_train = pd.concat([train,new_user]).fillna(0)
cols = new_train["user"].values.tolist()
cols = ["user "+ str(x) for x in cols]
corr_matrix = pd.DataFrame(cosine_similarity(new_train),columns = cols)
print ("Level 4")
line = corr_matrix.iloc[-1]

line = pd.DataFrame(line)
line["Users"] = line.index
user_id = line["Users"].iloc[-1]
line.columns = ["Similarity", "Users"]

final = line[(line["Similarity"]>0.9) & (line["Similarity"]<1)]["Users"].values.tolist()
final = [x for x in final if x != cols[-1]]
users =[float(x.strip("user "))for x in final] 
users = [str(int(x)) for x in users]
ok = df_500[df_500['user'].isin(users)]
recos = ok["genres"].values.tolist()
#print (users)
#
final_1=[]
for i in range(len(recos)):
    frames_genres=[]
    genre = recos[i].split(",")
    flat_list_g = [item for sublist in [i.split(':', 1) for i in genre] for item in sublist]
    flat_list_g = [i.lower() for i in flat_list_g]
    genre_1 = pd.DataFrame(flat_list_g[0::2]).T
    second_g = pd.DataFrame(flat_list_g[1::2]).T
    genre_df = pd.concat([genre_1,second_g],ignore_index = True)
    genre_df.columns = genre_df.iloc[0]
    genre_df = genre_df.drop([0],0)
    frames_genres.append(genre_df)
    final_df_g = pd.concat(frames_genres,join='outer', ignore_index=True)
    final_1.append(pd.DataFrame(final_df_g))
genre = pd.concat(final_1,join='outer', ignore_index=True)
genre = genre.fillna(0) 
#print (genre)
print (user_id)
genre = genre.apply(pd.to_numeric)
cols = genre.columns.tolist()
col_maxes =[]
for x in cols:
    col_maxes.append(genre[x].max())
d = dict(zip(cols,col_maxes))
import operator
f = sorted(d.items(), key=operator.itemgetter(1), reverse = True)
print ("lEVEL 5")
recommendations = []
for x in f[:4]:
    recommendations.append(x[0])
print (recommendations)







    



